In [1]:
# importing packages

from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS, LabelSet, Label
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Panel, Tabs, Button
from bokeh.models.glyphs import Text
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import matplotlib.pyplot as mplt
import matplotlib.lines as mlines
import time

output_notebook()

import numpy as np
import pandas as pd
from itertools import permutations
from gurobipy import *

Loading BokehJS ...

In [2]:
# settings

R = 9 # number of matrices max:8
M = 8 # number of rows max:20
N = 12 # number of columns max:20
K = 9 # number of partitions

F = N # flexibility of shapes
H = M # height of shapes
W = N # width of shapes

In [3]:
xls = pd.ExcelFile('data.xlsx')
df = [None]*9
df[0] = pd.read_excel(xls, 'Water Pipe (Cold Hot)', header=None)
df[1] = pd.read_excel(xls, 'High Conduit', header=None)
df[2] = pd.read_excel(xls, 'Rectangular Duct', header=None)
df[3] = pd.read_excel(xls, 'CAV-VAV', header=None)
df[4] = pd.read_excel(xls, 'M-Pipe-Wet & Connections', header=None)
df[5] = pd.read_excel(xls, 'Round Duct', header=None)
df[6] = pd.read_excel(xls, 'Cable Tray', header=None)
df[7] = pd.read_excel(xls, 'Conduit Runs', header=None)
df[8] = pd.read_excel(xls, 'Electrical CAV-VAV', header=None)

In [4]:
b = [[]] * R
for r in range(R):
    b[r] = np.array(df[r])

In [5]:
# parameters

np.random.seed(2)

a = [[]] * R

for r in range(R):
    a[r] = np.zeros((M, N+1))
    
for r in range(R):
    for i in range(M):
        for j in range(1, N+1):
            a[r][i][j] = a[r][i][j-1] + b[r][i][j-1]
            
Si = np.array(range(0,M,1))
Sj = np.array(range(0,N+1,1))
Sk = np.array(range(0,K+1,1))
Sr = np.array(range(0,R,1))

In [6]:
TimeTotal = time.time()

# model
BCP = Model('BCP_q model')

# variables
x = BCP.addVars(Si, Sj, Sk, lb=0.0, ub=1.0, vtype='B', name="X")
ux = BCP.addVars(Si, Sj, Sk, lb=0.0, vtype='C', name="X")
y = BCP.addVars(Si, Sk, lb=0.0, ub=1.0, vtype='B', name="Y")
B = BCP.addVars(Sk, lb=0.0, ub=1.0, vtype='B', name="B")
BB = BCP.addVars(Sk, lb=0.0, ub=1.0, vtype='B', name="B")
s = BCP.addVars(Si, Sk, lb=0.0, ub=1.0, vtype='B', name="S")
z = BCP.addVars(Sr, Sk, lb=0.0, vtype='C', name="Z")
OBJ = BCP.addVar(lb=0.0, vtype='C', name="OBJECTIVE")

# constraint 0
BCP.addConstrs(x[i,0,0] == 1 for i in Si)
BCP.addConstrs(x[i,N,K] == 1 for i in Si)
BCP.update()

# constraint 1
BCP.addConstrs((
    quicksum(x[i, j, k] for j in Sj) == 1 
    for i in Si
    for k in Sk),
    name = "exactly k cuts per row")
BCP.update()

# constraint 2c
BCP.addConstrs((
    ux[i,j,k] >= x[i,j,k] + y[i,k] - 1
    for n in Sj
    for i in Si
    for k in Sk),
    name = "ux")
BCP.update()

# constraint 2''
BCP.addConstrs((
    quicksum(x[i,j,k-1] - x[i,j,k] for j in Sj[:n])>= ux[i,n,k]
    for n in Sj[1:]
    for i in Si
    for k in Sk[1:]),
    name = "Order of cuts SSS")
BCP.update()

# # constraint 3
BCP.addConstrs((
    W * y[i, k] >= quicksum(j*(x[i,j,k]-x[i,j,k-1]) for j in Sj) 
    for i in Si
    for k in Sk[1:]),
    name = "Openness_1")
BCP.addConstrs((
    y[i, k] <= quicksum(j*(x[i,j,k]-x[i,j,k-1]) for j in Sj) 
    for i in Si
    for k in Sk[1:]),
    name = "Openness_2")
BCP.update()

# # constraint 3''
BCP.addConstrs((
    quicksum(x[i,j,k-1] for j in Sj[:n]) <= quicksum(x[i,j,k] for j in Sj[:n]) + y[i, k]
    for n in Sj[1:]
    for i in Si
    for k in Sk[1:]),
    name = "Openness_1 SSS")
BCP.update()

# constraint 4
BCP.addConstrs((
    s[i,k] + y[i-1,k] >= y[i,k] 
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Starts")
BCP.addConstrs((
    s[0,k] >= y[0,k]
    for k in Sk[1:]),
    name = "Starts_0")
BCP.update()

# constraint 5
BCP.addConstrs((
    quicksum(s[i,k] for i in Si) == 1
    for k in Sk[1:]),
    name = "One start for each partition")
BCP.update()

# constraint 5'
BCP.addConstrs((
    quicksum(s[i,k]*(nn+1) - y[i,k] for i in Si[:nn+1]) >= 0
    for nn in Si
    for k in Sk[1:]),
    name = "Maybe a cut")
BCP.update()

# constraint 6
BCP.addConstrs((
    quicksum(j*(x[i-1,j,k-1]) for j in Sj) <= quicksum(j*(x[i,j,k]) for j in Sj) - 1 + M * (2-y[i-1,k]-y[i,k])
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_1")
BCP.addConstrs((
    quicksum(j*(x[i,j,k-1]) for j in Sj) <= quicksum(j*(x[i-1,j,k]) for j in Sj) - 1 + M * (2-y[i-1,k]-y[i,k])
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_2")
BCP.update()

# constraint 6''a
BCP.addConstrs((
    quicksum(x[i-1,j,k-1] for j in Sj[:n]) >= quicksum(x[i,j,k] for j in Sj[:n+1]) + y[i-1,k] + y[i,k] - 2
    for n in Sj[1:-1]
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_1 SSS")

# constraint 6''b
BCP.addConstrs((
    quicksum(x[i,j,k-1] for j in Sj[:n]) >= quicksum(x[i-1,j,k] for j in Sj[:n+1]) + y[i-1,k] + y[i,k] - 2
    for n in Sj[1:-1]
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_2 SSS")
BCP.update()




#-----------------------------------



BCP.addConstrs((
    quicksum(x[i-1,jj,k] for jj in Sj[max(j-F,0):min(j+F+1,N+1)]) >= x[i,j,k] + y[i-1,k] + y[i,k] - 2
    for j in Sj
    for i in Si[1:]
    for k in Sk),
    name = "Vicinity_1 FLEX")


BCP.addConstrs((
    quicksum(x[i,jj,k] for jj in Sj[max(j-F,0):min(j+F+1,N+1)]) >= x[i-1,j,k] + y[i-1,k] + y[i,k] - 2
    for j in Sj
    for i in Si[1:]
    for k in Sk),
    name = "Vicinity_2 FLEX")



BCP.addConstrs((
    quicksum(x[i-1,jj,k-1] for jj in Sj[max(j-F,0):min(j+F+1,N+1)]) >= x[i,j,k-1] + y[i-1,k] + y[i,k] - 2
    for j in Sj
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_3 FLEX")


BCP.addConstrs((
    quicksum(x[i,jj,k-1] for jj in Sj[max(j-F,0):min(j+F+1,N+1)]) >= x[i-1,j,k-1] + y[i-1,k] + y[i,k] - 2
    for j in Sj
    for i in Si[1:]
    for k in Sk[1:]),
    name = "Vicinity_4 FLEX")



#-----------------------------------




BCP.addConstrs((
    quicksum(y[i,k] for i in Si) <= H
    for k in Sk),
    name = "Height")


# constraint 7
BCP.addConstrs((
    z[r,k] == quicksum(a[r][i][j]*(x[i,j,k]-x[i,j,k-1]) for j in Sj for i in Si)
    for r in Sr
    for k in Sk[1:]),
    name = "Sum of the partition")

# constraint 7''
BCP.addConstrs((
    OBJ >= z[r,k]
    for r in Sr
    for k in Sk[1:]),
    name = "OBJ on Sum of the partition")

BCP.update()

# objective
BCP.setObjective(OBJ, GRB.MINIMIZE)

# solution
BCP.optimize()

ttime = time.time()-TimeTotal
print('\n\n\nTotal - Time = {}     Obj = {}'.format(round(ttime, 1), round(BCP.ObjVal, 2)))

Academic license - for non-commercial use only
Optimize a model with 8303 rows, 2351 columns and 122404 nonzeros
Variable types: 1131 continuous, 1220 integer (1220 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 5266 rows and 1247 columns
Presolve time: 0.19s
Presolved: 3037 rows, 1104 columns, 44573 nonzeros
Variable types: 79 continuous, 1025 integer (1016 binary)
Found heuristic solution: objective 16.8852000
Found heuristic solution: objective 15.7173000
Found heuristic solution: objective 15.5463000

Root relaxation: objective 1.989767e+00, 1186 iterations, 0.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.98977    0  120   15.54630    1.98977  87.2%     -    0s
H    0     0                       3.2000000    1.989

 201879 39942 infeasible   67         2.04050    1.98977  2.49%   112  301s
 207724 41128    2.00345   62   69    2.04050    1.98977  2.49%   112  307s
 209453 41171    1.98977   67  106    2.04050    1.98977  2.49%   112  310s
 216011 42678    1.98977   68  113    2.04050    1.98977  2.49%   112  315s
 220468 43583    1.98977   86   70    2.04050    1.98977  2.49%   112  321s
 227260 45254 infeasible   87         2.04050    1.98977  2.49%   112  327s
 229395 45550    1.98977   62   63    2.04050    1.98977  2.49%   112  331s
 235320 46663    1.98977   73  113    2.04050    1.98977  2.49%   112  336s
 240476 47614    2.01770   62   64    2.04050    1.98977  2.49%   112  342s
 243933 48461    1.98977   65   96    2.04050    1.98977  2.49%   111  345s
 249424 49337    1.98977   73  131    2.04050    1.98977  2.49%   111  351s
 254615 50726     cutoff   87         2.04050    1.98977  2.49%   111  356s
 259070 51807    1.98977   70  122    2.04050    1.98977  2.49%   111  362s
 262593 5279

 637123 109426    1.98977   70   93    2.02220    1.98977  1.60%   114  800s
 643372 110480    1.99909   89   96    2.02220    1.98977  1.60%   113  805s
 649025 111231    1.98977   63   65    2.02220    1.98977  1.60%   113  810s
 651551 111853    1.98977   70   77    2.02220    1.98977  1.60%   113  815s
 654615 111962 infeasible   72         2.02220    1.98977  1.60%   113  820s
 660426 112980 infeasible   82         2.02220    1.98977  1.60%   113  825s
 665765 113916 infeasible   61         2.02220    1.98977  1.60%   113  831s
 669059 114025    1.98977   65  118    2.02220    1.98977  1.60%   113  836s
 675335 115301 infeasible   83         2.02220    1.98977  1.60%   113  841s
 681350 116631     cutoff   80         2.02220    1.98977  1.60%   113  847s
 685112 116853    2.01903   71   46    2.02220    1.98977  1.60%   113  851s
 690741 117806    2.02219   83   53    2.02220    1.98977  1.60%   113  857s
 694010 117763    1.98977   83   96    2.02220    1.98977  1.60%   113  862s

 1109306 184641    1.98977   65  102    2.02220    1.98977  1.60%   110 1330s
 1112420 185039 infeasible  106         2.02220    1.98977  1.60%   110 1336s
 1118743 185843    1.99325   87   72    2.02220    1.98977  1.60%   110 1341s
 1124514 186902 infeasible   91         2.02220    1.98977  1.60%   110 1348s
 1124753 186589    1.98977   54  101    2.02220    1.98977  1.60%   110 1351s
 1130606 187437 infeasible   84         2.02220    1.98977  1.60%   110 1356s
 1136192 188240 infeasible   63         2.02220    1.98977  1.60%   110 1361s
 1138870 188942    2.01114   95   88    2.02220    1.98977  1.60%   110 1366s
 1142578 189089    1.99620   93  153    2.02220    1.98977  1.60%   110 1371s
 1148471 189690    1.98977   80  125    2.02220    1.98977  1.60%   110 1376s
 1152276 190260 infeasible   91         2.02220    1.98977  1.60%   110 1382s
 1155379 190720    1.98977   55  131    2.02220    1.98977  1.60%   110 1385s
 1161251 191763    2.00926   65   84    2.02220    1.98977  1.60

 1568612 250464    2.00181   64   78    2.02220    1.98977  1.60%   110 1865s
 1574051 251226 infeasible   68         2.02220    1.98977  1.60%   110 1870s
 1576940 251996    1.98977   63  139    2.02220    1.98977  1.60%   110 1875s
 1580319 252219 infeasible   71         2.02220    1.98977  1.60%   111 1880s
 1586659 253507 infeasible   80         2.02220    1.98977  1.60%   111 1887s
 1588314 253547    1.99033   73   91    2.02220    1.98977  1.60%   111 1890s
 1595011 255025 infeasible   69         2.02220    1.98977  1.60%   110 1895s

Cutting planes:
  Gomory: 1
  Cover: 1
  MIR: 1
  StrongCG: 3
  Flow cover: 6
  GUB cover: 1
  Inf proof: 101
  Zero half: 2

Explored 1597912 nodes (176564918 simplex iterations) in 1897.59 seconds
Thread count was 24 (of 24 available processors)

Solution count 10: 2.0222 2.0222 2.0222 ... 2.0369

Solve interrupted
Best objective 2.022199862724e+00, best bound 1.989766666667e+00, gap 1.6039%



Total - Time = 1898.7     Obj = 2.02


In [7]:
# drawing Matrix

c_m = np.zeros(shape=(M,K+1)).astype(int)


for i in range(M):
    for j in range(N+1):
        for k in range(K+1):
            c_m[i,k] += round(x[i,j,k].x*j)


col = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',\
           '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',\
           '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']


plt = figure(plot_width=500, plot_height=round(500*M/N))


for i in range(0,M):
    for k in range(1,K+1):
        plt.quad(top=[-i], bottom=[-i-1], left=[c_m[i,k-1]], right=[c_m[i,k]], color=col[(k-1)%20], alpha = 0.4)



# for i in range(-m,0):
#     for j in range(1,n+1):
#         plt.quad(top=[i+0.5], bottom=[i-00.5], left=[j-0.5], right=[j+0.5], color=col[a_m[-i-1,j-1]%20])
   
    
for i in range(0,M+1):
    plt.line([0, N], [-i, -i], line_color='black', line_width=1, alpha = 1)
for i in range(0,N+1):
    plt.line([i, i], [0, -M], line_color='black', line_width=1, alpha = 1)


print(BCP.ObjVal)
pltName = "TEST.html"
output_file(pltName)
show(plt)

2.0221998627242685


In [8]:
for v in BCP.getVars():
    if (v.x != 0):
        print(v.VarName, round(v.x,2))

X[0,0,0] 1.0
X[0,0,1] 1.0
X[0,0,2] 1.0
X[0,0,3] 1.0
X[0,9,4] 1.0
X[0,12,5] 1.0
X[0,12,6] 1.0
X[0,12,7] 1.0
X[0,12,8] 1.0
X[0,12,9] 1.0
X[1,0,0] 1.0
X[1,5,1] 1.0
X[1,5,2] 1.0
X[1,5,3] 1.0
X[1,7,4] 1.0
X[1,12,5] 1.0
X[1,12,6] 1.0
X[1,12,7] 1.0
X[1,12,8] 1.0
X[1,12,9] 1.0
X[2,0,0] 1.0
X[2,1,1] 1.0
X[2,7,2] 1.0
X[2,7,3] 1.0
X[2,7,4] 1.0
X[2,7,5] 1.0
X[2,7,6] 1.0
X[2,12,7] 1.0
X[2,12,8] 1.0
X[2,12,9] 1.0
X[3,0,0] 1.0
X[3,2,1] 1.0
X[3,3,2] 1.0
X[3,8,3] 1.0
X[3,8,4] 1.0
X[3,8,5] 1.0
X[3,8,6] 1.0
X[3,10,7] 1.0
X[3,12,8] 1.0
X[3,12,9] 1.0
X[4,0,0] 1.0
X[4,2,1] 1.0
X[4,3,2] 1.0
X[4,4,3] 1.0
X[4,4,4] 1.0
X[4,4,5] 1.0
X[4,6,6] 1.0
X[4,6,7] 1.0
X[4,12,8] 1.0
X[4,12,9] 1.0
X[5,0,0] 1.0
X[5,1,1] 1.0
X[5,1,2] 1.0
X[5,1,3] 1.0
X[5,1,4] 1.0
X[5,1,5] 1.0
X[5,7,6] 1.0
X[5,7,7] 1.0
X[5,7,8] 1.0
X[5,12,9] 1.0
X[6,0,0] 1.0
X[6,6,1] 1.0
X[6,6,2] 1.0
X[6,6,3] 1.0
X[6,6,4] 1.0
X[6,6,5] 1.0
X[6,8,6] 1.0
X[6,8,7] 1.0
X[6,8,8] 1.0
X[6,12,9] 1.0
X[7,0,0] 1.0
X[7,0,1] 1.0
X[7,0,2] 1.0
X[7,0,3] 1.0
X[7,0,4] 1.0
X[7,0

In [9]:
R_name = ['Water Pipe (Cold Hot)', 'High Conduit', 'Rectangular Duct', 'CAV-VAV', 'M-Pipe-Wet & Connections',\
          'RoundDuct', 'CableTray', 'Conduit Runs', 'Electrical CAV-VAV']

col_rct = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',\
       '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',\
       '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

col = ['#000075','#42d4f4','#f58231','#8A2BE2','#A52A2A','#f032e6','#4363d8','#808000', 'green']



partitions = []
for i in range(1,K+1):
    partitions.append("Partition_" + str(i))

data = {}
data['prtitions'] = partitions
for r in range(R):
    data[R_name[r]] = []
    for k in range(K):
        data[R_name[r]].append(z[r,k+1].x)

w = 2.8/(5*R-1)
source = ColumnDataSource(data=data)

p = figure(x_range=partitions, y_range=(0, BCP.ObjVal*1.5), plot_height=350, plot_width=1250,\
           title="TAKT planning result", tools="save")

for k in range(K):
    p.quad(top=[BCP.ObjVal*1.3], bottom=[0], left=[k], right=[k+1], color=col_rct[k], alpha = 0.2)    

for r in range(R):
    p.vbar(x=dodge('prtitions', w*5/4*r-0.35+w/2, range=p.x_range), top=R_name[r], width=w, source=source,
       color=col[r], legend=value(R_name[r]))

p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)